In [1]:
from PIL import Image
import numpy as np

In [4]:

def is_flag_of_indonesia_or_poland(image_path):
    # Load and preprocess the image
    image = Image.open(image_path).convert("RGB").resize((400, 300))
    pixels = np.array(image)
    height, width, _ = pixels.shape

    # Color detection functions
    def is_red(pixel):
        r, g, b = pixel
        return r > 150 and g < 100 and b < 100

    def is_white(pixel):
        r, g, b = pixel
        return r > 200 and g > 200 and b > 200

    # Create masks for red and white pixels
    red_mask = np.array([[is_red(p) for p in row] for row in pixels])
    white_mask = np.array([[is_white(p) for p in row] for row in pixels])

    # Combine masks and determine the bounding box
    combined_mask = red_mask | white_mask
    rows, cols = np.where(combined_mask)
    if len(rows) == 0 or len(cols) == 0:
        return "Neither"

    # Crop the flag region
    top, bottom = min(rows), max(rows)
    left, right = min(cols), max(cols)
    cropped_flag = image.crop((left, top, right, bottom)).resize((200, 100))
    flag_pixels = np.array(cropped_flag)

    # Analyze color distribution in halves
    mid = flag_pixels.shape[0] // 2
    top_half, bottom_half = flag_pixels[:mid], flag_pixels[mid:]

    def count_colors(section):
        red_count = sum(is_red(pixel) for row in section for pixel in row)
        white_count = sum(is_white(pixel) for row in section for pixel in row)
        return red_count, white_count

    red_top, white_top = count_colors(top_half)
    red_bottom, white_bottom = count_colors(bottom_half)

    # Determine flag type
    if red_top > white_top and white_bottom > red_bottom:
        return "Indonesia"
    elif white_top > red_top and red_bottom > white_bottom:
        return "Poland"
    else:
        return "Neither"

# Example usage
result = is_flag_of_indonesia_or_poland('/home/majistor/Desktop/flag.jpg')
print(result)

Indonesia
